In [1]:
import os
import requests
import time
import fitz
import tempfile
from pathlib import Path
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

# 환경변수에서 API 키 가져오기
API_KEY = os.getenv("UPSTAGE_API_KEY")

In [2]:
# PDF 분할 함수
def split_pdf(input_path, max_pages=50):
    doc = fitz.open(input_path)
    total_pages = len(doc)
    print("문서의 총 페이지 수:", total_pages)
    for start in range(0, total_pages, max_pages):
        end = min(start + max_pages, total_pages)
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as temp_file:
            temp_doc = fitz.open()
            temp_doc.insert_pdf(doc, from_page=start, to_page=end-1)
            temp_doc.save(temp_file.name)
            temp_doc.close()
            yield temp_file.name

In [3]:
pdf_path = "./raw_docs/상품3/판매약관_신한(간편가입)든든한실속종신보험(무배당, 보증비용부과형)[해약환급금 미지급형]_240903.pdf"

url = "https://api.upstage.ai/v1/document-ai/async/layout-analysis"
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

all_extracted_text = []

# 전체 페이지 수 확인
total_pages = len(fitz.open(pdf_path))
print(f"총 페이지 수: {total_pages}")

processed_pages = 0
total_start_time = time.time()
total_page_processing_time = 0

for i, split_pdf_path in enumerate(split_pdf(pdf_path), 1):
    split_start_time = time.time()
    print(f"\n처리 중인 분할 PDF {i}")
    
    with open(split_pdf_path, "rb") as file:
        files = {"document": file}
        data = {"ocr": "true"}
        
        print("API 요청 제출 중...")
        response = requests.post(url, headers=headers, files=files, data=data)
        response.raise_for_status()
        request_id = response.json()["request_id"]
    
    print("결과 대기 중...")
    status_url = f"https://api.upstage.ai/v1/document-ai/requests/{request_id}"
    while True:
        response = requests.get(status_url, headers=headers)
        response.raise_for_status()
        status_data = response.json()
            
        if status_data["status"] == "completed":
            break
        elif status_data["status"] == "failed":
            raise Exception(f"처리 실패: {status_data['failure_message']}")
            
        print(".", end="", flush=True)
        time.sleep(10)
    
    print("\n결과 다운로드 및 처리 중...")
    extracted_text = []
    for batch in status_data["batches"]:
        download_url = batch["download_url"]
        batch_response = requests.get(download_url)
        batch_response.raise_for_status()
        batch_data = batch_response.json()
        
        if "elements" in batch_data:
            pages = {}
            for element in batch_data["elements"]:
                page_num = element.get("page", 0)
                if page_num not in pages:
                    pages[page_num] = []
                pages[page_num].append(element)
            
            for page_num, elements in sorted(pages.items()):
                page_start_time = time.time()
                processed_pages += 1
                page_content = "\n".join([element["text"] for element in elements if "text" in element])
                extracted_text.append(page_content)
                extracted_text.append(f"\n\n--- End of Page {page_num} ---\n\n")
                page_end_time = time.time()
                page_duration = page_end_time - page_start_time
                total_page_processing_time += page_duration
                print(f"페이지 {processed_pages}/{total_pages} 처리 완료 (소요 시간: {page_duration:.2f}초)")
        else:
            print("예상치 못한 배치 데이터 구조. 'elements' 키를 찾을 수 없습니다.")
            continue
                
    all_extracted_text.extend(extracted_text)

    os.unlink(split_pdf_path)
    split_end_time = time.time()
    split_duration = split_end_time - split_start_time
    print(f"분할 PDF {i} 처리 완료 (소요 시간: {split_duration:.2f}초)")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
avg_page_duration = total_duration / total_pages

print(f"\n텍스트 추출 완료")
print(f"총 소요 시간: {total_duration:.2f}초")
print(f"평균 페이지 처리 시간: {avg_page_duration:.2f}초")

# 추출된 텍스트를 파일로 저장
input_dir = Path(pdf_path).parent
output_file = input_dir / f"{Path(pdf_path).stem}_extracted.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("".join(all_extracted_text))

print(f"\n추출된 텍스트가 다음 위치에 저장되었습니다: {output_file}")
print("추출된 텍스트 (처음 500자):")
print("".join(all_extracted_text)[:500] + "...")

총 페이지 수: 342
문서의 총 페이지 수: 342

처리 중인 분할 PDF 1
API 요청 제출 중...
결과 대기 중...
..
결과 다운로드 및 처리 중...
페이지 1/342 처리 완료 (소요 시간: 0.00초)
페이지 2/342 처리 완료 (소요 시간: 0.00초)
페이지 3/342 처리 완료 (소요 시간: 0.00초)
페이지 4/342 처리 완료 (소요 시간: 0.00초)
페이지 5/342 처리 완료 (소요 시간: 0.00초)
페이지 6/342 처리 완료 (소요 시간: 0.00초)
페이지 7/342 처리 완료 (소요 시간: 0.00초)
페이지 8/342 처리 완료 (소요 시간: 0.00초)
페이지 9/342 처리 완료 (소요 시간: 0.00초)
페이지 10/342 처리 완료 (소요 시간: 0.00초)
페이지 11/342 처리 완료 (소요 시간: 0.00초)
페이지 12/342 처리 완료 (소요 시간: 0.00초)
페이지 13/342 처리 완료 (소요 시간: 0.00초)
페이지 14/342 처리 완료 (소요 시간: 0.00초)
페이지 15/342 처리 완료 (소요 시간: 0.00초)
페이지 16/342 처리 완료 (소요 시간: 0.00초)
페이지 17/342 처리 완료 (소요 시간: 0.00초)
페이지 18/342 처리 완료 (소요 시간: 0.00초)
페이지 19/342 처리 완료 (소요 시간: 0.00초)
페이지 20/342 처리 완료 (소요 시간: 0.00초)
페이지 21/342 처리 완료 (소요 시간: 0.00초)
페이지 22/342 처리 완료 (소요 시간: 0.00초)
페이지 23/342 처리 완료 (소요 시간: 0.00초)
페이지 24/342 처리 완료 (소요 시간: 0.00초)
페이지 25/342 처리 완료 (소요 시간: 0.00초)
페이지 26/342 처리 완료 (소요 시간: 0.00초)
페이지 27/342 처리 완료 (소요 시간: 0.00초)
페이지 28/342 처리 완료 (소요 시간: 0.00초)
페이지 29/342 처리 완료 (소요